# Listing objects in an Environment

All objects in an environment are cataloged by the Coordinating Nodes. The [`CN.listObjects()`](https://dataone-architecture-documentation.readthedocs.io/en/latest/apis/CN_APIs.html#CNRead.listObjects) method provides access to this list. Since there can be a large number of objects in an environment, the page size of a response is limited, and iterating through all the objects requires paging requests.

In the following example, the page size is limited to 5 entries:

In [ ]:
# notebook utils contains some convenience methods, see notebook_utils/__init__.py
import notebook_utils as nbu

# Import the library and create a client instance
from d1_client import baseclient_2_0

cn_base_url = "https://cn.dataone.org/cn"
client = baseclient_2_0.DataONEBaseClient_2_0(cn_base_url)
response = client.listObjects( count=5, start=0 )

print("XML Response:")
print(nbu.asXml(response, max_lines=25))

Show the response, printing out each entry.

In [ ]:
def printResults(response):
    print(f"Total objects: {response.total} Start: {response.start}  Page size: {response.count}\n")
    counter = response.start
    for entry in response.objectInfo:
        print(f"{counter:08d}: ")
        print(f"            PID: {nbu.propertyStr(entry.identifier)}")
        print(f"       formatId: {nbu.propertyStr(entry.formatId)}")
        print(f"           size: {nbu.propertyStr(entry.size)}")
        print(f"  date_modified: {nbu.propertyStr(entry.dateSysMetadataModified)}")
        print("")
        counter += 1

printResults(response)

## Add a date filter

Add a `fromDate` parameter, so `listObjects` will respond with the list of entries that were modified between one day ago and now.

In [ ]:
import dateparser

start_date = dateparser.parse('last week UTC', 
                              settings={'RETURN_AS_TIMEZONE_AWARE': True})

response = client.listObjects( 
    count=5, 
    start=0,
    fromDate=start_date
)

printResults( response )


## Paging the response

The server will limit the total number of records returned. When requesting large sets of entries, the 
response will need to be examined to determine if additional pages of results should be requested.

In [ ]:
start_date = dateparser.parse('two weeks ago UTC', 
                              settings={'RETURN_AS_TIMEZONE_AWARE': True})
end_date = dateparser.parse('one week ago UTC', 
                              settings={'RETURN_AS_TIMEZONE_AWARE': True})
max_to_retrieve = 25  # limit total numbe of entries to download

params = {
    "count": 3, #specify a small page size
    "start": 0,
    "fromDate": start_date,
    "toDate": end_date,
}
response = client.listObjects( **params )

if max_to_retrieve > response.total:
    max_to_retrieve = response.total

printResults( response )

num_retrieved = response.count
while num_retrieved < max_to_retrieve:
    params['start'] += response.count
    response = client.listObjects( **params )
    num_retrieved += response.count
    printResults( response )
    